# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [33]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [34]:
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,Unnamed: 0,city,lat,lon,temp,hum,clo,win,country,date
0,0,ushuaia,-54.8000,-68.3000,37.06,87,75,16.11,AR,1628887320
1,1,hithadhoo,-0.6000,73.0833,81.72,75,97,13.09,MV,1628887143
2,2,hermanus,-34.4187,19.2345,49.80,80,1,12.59,ZA,1628887135
3,3,vaini,-21.2000,-175.2000,66.36,88,40,6.91,TO,1628887191
4,4,upernavik,72.7868,-56.1549,43.95,95,100,6.76,GL,1628887324


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [35]:
#Configue gmaps
gmaps.configure(api_key = g_key)
gmaps.figure()

Figure(layout=FigureLayout(height='420px'))

In [36]:
location = city_data_df[["lat","lon"]]
humidity = city_data_df["hum"]


In [37]:
#Addition of the heatmap
fig = gmaps.figure()

#Creating heat layer
heat_layer = gmaps.heatmap_layer(location, weights= humidity, dissipating=False, max_intensity=300,point_radius=10)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [38]:
cities_data_df= city_data_df.loc[(city_data_df["temp"]< 80)&(city_data_df["temp"]> 70)& (city_data_df ["win"]<10)\
                                                      & (city_data_df["clo"]==0)].dropna()
cities_data_df

,Unnamed: 0,city,lat,lon,temp,hum,clo,win,country,date
126,126,vila velha,-20.3297,-40.2925,75.15,78,0,8.05,BR,1628887064
240,240,dawlatabad,36.4141,64.9053,78.10,24,0,3.18,AF,1628887665
243,243,fethiye,36.6217,29.1164,78.67,38,0,5.93,TR,1628887669
314,314,harsin,34.2721,47.5861,76.01,22,0,1.23,IR,1628887774
451,451,kilis,36.7161,37.1150,75.43,49,0,5.57,TR,1628887968
497,497,kamyaran,34.7956,46.9355,76.95,20,0,2.59,IR,1628888032
532,532,marovoay,-16.1000,46.6333,75.34,76,0,3.60,MG,1628888082
556,556,bardiyah,31.7561,25.0865,79.95,71,0,8.03,LY,1628888121


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [39]:
#New datafram hotel_df
hotel_df= cities_data_df[["city","lat", "lon", "country"]].copy()
hotel_df['Hotel Name'] = ""
hotel_df

,city,lat,lon,country,Hotel Name
126,vila velha,-20.3297,-40.2925,BR,
240,dawlatabad,36.4141,64.9053,AF,
243,fethiye,36.6217,29.1164,TR,
314,harsin,34.2721,47.5861,IR,
451,kilis,36.7161,37.1150,TR,
497,kamyaran,34.7956,46.9355,IR,
532,marovoay,-16.1000,46.6333,MG,
556,bardiyah,31.7561,25.0865,LY,


In [55]:
#Setting parameters

params = {
      "radius":5000,
      "type": "hotel",
      "Key"  : g_key,
    
}

for index, row in hotel_df.iterrows():
    lat = row["lat"]
    lon = row["lon"]
    
    params['keyword'] = f'{lat},{lon}'
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
 #Hotel result into DataFrame
    try:
    
      hotel_df.loc[index,"Hotel Name"]=results[0]["Hotel Name"]
    
    except (KeyError, IndexError): 
    
      print("Missing field/result... skipping.")
   
    print("---------------")

hotel_df

Missing field/result... skipping.
---------------
Missing field/result... skipping.
---------------
Missing field/result... skipping.
---------------
Missing field/result... skipping.
---------------
Missing field/result... skipping.
---------------
Missing field/result... skipping.
---------------
Missing field/result... skipping.
---------------
Missing field/result... skipping.
---------------


,city,lat,lon,country,Hotel Name
126,vila velha,-20.3297,-40.2925,BR,
240,dawlatabad,36.4141,64.9053,AF,
243,fethiye,36.6217,29.1164,TR,
314,harsin,34.2721,47.5861,IR,
451,kilis,36.7161,37.1150,TR,
497,kamyaran,34.7956,46.9355,IR,
532,marovoay,-16.1000,46.6333,MG,
556,bardiyah,31.7561,25.0865,LY,


In [61]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>city</dt><dd>{city}</dd>
<dt>country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [64]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))